In [10]:
import cv2
import mediapipe as mp
import json
import os
from glob import glob
from PIL import Image
import numpy as np

# Mediapipe Pose 모델 로드
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

selected_landmarks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 22, 23, 24, 25, 26, 27, 28]

data = {
    "frames": [],
    "type": "561",
    "type_info": {
        "key": "561",
        "type": "맨몸 운동",
        "pose": "엎드린자세",
        "exercise": "푸시업",
        "conditions": [
            {"condition": "척추의 중립", "value": True},
            {"condition": "이완시 팔꿈치 90도", "value": True},
            {"condition": "가슴의 충분한 이동", "value": True},
            {"condition": "손의 위치 가슴 중앙 여부", "value": True},
            {"condition": "고개 젖힘/숙임 여부", "value": True}
        ],
        "description": "정자세"
    }
}

# 부모 폴더 (서브 폴더가 존재하는 폴더)
parent_folder = "D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/gradu"

# 서브 폴더 목록 가져오기 (최대 5개 선택)
subfolders = sorted([f.path for f in os.scandir(parent_folder) if f.is_dir()])[:5]

# Mediapipe Pose 모델 실행
with mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False) as pose:
    for folder_idx, subfolder in enumerate(subfolders):
        view_name = f"view{folder_idx+1}"  # view1, view2, ..., view5

        # 서브 폴더 내 이미지 파일 가져오기
        image_files = sorted(glob(os.path.join(subfolder, "*.jpg")))

        if not image_files:
            print(f"⚠️ {subfolder} 에서 이미지를 찾을 수 없습니다.")
            continue

        print(f"📂 Processing folder for {view_name}: {subfolder}")

        try:
            # 첫 번째 이미지만 사용
            img_file = image_files[0]

            # PIL을 사용하여 이미지 로드
            image = Image.open(img_file).convert("RGB")
            image_np = np.array(image)

            # Mediapipe에 입력할 수 있도록 배열 변환 (H, W, C 순서 유지)
            results = pose.process(image_np)

            # 키포인트 좌표 저장할 딕셔너리 (정규화된 값 유지)
            keypoints = {}

            if results.pose_landmarks:
                for idx, landmark in enumerate(results.pose_landmarks.landmark):
                    keypoints[f"Point_{idx}"] = {
                        "x": landmark.x,  # 정규화된 값 (0~1)
                        "y": landmark.y   # 정규화된 값 (0~1)
                    }

            # JSON 구조에 view 추가
            frame_data = {
                "pts": keypoints,
                "active": "Yes" if results.pose_landmarks else "No",
                "img_key": img_file
            }

            # 한 프레임 내에 5개의 view 포함
            if len(data["frames"]) == 0:
                data["frames"].append({})  # 첫 프레임 생성
            data["frames"][0][view_name] = frame_data

        except Exception as e:
            print(f"❌ 이미지 처리 오류 ({subfolder}): {e}")
            continue

# JSON 파일 저장
json_output_path = "D:/Studying/gradu/013.피트니스자세/1.Training/gradu/body09-1-561.json"
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print(f"✅ 5개 View JSON 파일이 {json_output_path}에 저장되었습니다.")


📂 Processing folder for view1: D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/gradu\561-1-3-27-Z18_A
📂 Processing folder for view2: D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/gradu\561-1-3-27-Z18_B
📂 Processing folder for view3: D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/gradu\561-1-3-27-Z18_C
📂 Processing folder for view4: D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/gradu\561-1-3-27-Z18_D
📂 Processing folder for view5: D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/gradu\561-1-3-27-Z18_E
✅ 5개 View JSON 파일이 D:/Studying/gradu/013.피트니스자세/1.Training/gradu/body09-1-561.json에 저장되었습니다.


In [30]:
import cv2

test_image_path = image_files[0]  # 첫 번째 이미지 파일 테스트

print(f"🔍 테스트 이미지: {test_image_path}")

image = cv2.imread(test_image_path)
if image is None:
    print("❌ OpenCV에서 이미지를 읽을 수 없습니다.")
else:
    print("✅ 이미지 로드 성공")
    cv2.imshow("Test Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

from PIL import Image

try:
    img = Image.open(test_image_path)
    img.show()
    print("✅ PIL로 이미지 로드 성공")
except Exception as e:
    print("❌ PIL에서 이미지 로드 실패:", e)

🔍 테스트 이미지: D:/Studying/gradu/013.피트니스자세/1.Training/원시데이터/body_09/Day35_201107_F/1/A/561-1-3-27-Z18_A\561-1-3-27-Z18_A-0000001.jpg
❌ OpenCV에서 이미지를 읽을 수 없습니다.
✅ PIL로 이미지 로드 성공
